Data Prep

In [1]:
import sqlite3
import pandas as pd

# Load dataset
df = pd.read_csv("logistics_forecasting_data.csv")

# Connect to SQLite
conn = sqlite3.connect("logistics.db")

# Store in SQLite
df.to_sql("shipments", conn, if_exists="replace", index=False)


100

High-Cost Shipments

In [5]:
query = """
SELECT *, 
       (SELECT AVG(Total_Cost) FROM shipments) AS Avg_Cost
FROM shipments
WHERE Total_Cost > 1.5 * (SELECT AVG(Total_Cost) FROM shipments);
"""
pd.read_sql(query, conn)

,Shipment_ID,Order_Date,Warehouse_Latitude,Warehouse_Longitude,Destination_Latitude,Destination_Longitude,Shipping_Distance_km,Shipping_Cost,Fuel_Cost,Labor_Cost,Total_Cost,Delivery_Time_Days,Weather_Impact,Traffic_Delay_Hours,Damaged_Shipment,Shipment_Anomaly,Avg_Cost
0,2032,2023-02-01,33.410482,-111.115688,40.079113,-87.107694,3819.48,499.86,179.57,246.23,925.66,9,None,28.72,1,None,523.8307
1,2033,2023-02-02,31.301032,-115.205385,42.307117,-124.280326,3110.18,498.39,104.78,202.91,806.08,5,Moderate,22.06,0,Delayed,523.8307
2,2036,2023-02-05,46.167947,-107.071883,46.673563,-122.963560,3519.80,473.49,92.25,264.39,830.13,4,Moderate,0.11,0,None,523.8307
3,2052,2023-02-21,45.502656,-110.417524,36.552932,-82.175508,2814.10,429.05,109.74,282.55,821.34,11,Moderate,30.59,0,None,523.8307
4,2072,2023-03-13,33.974314,-119.336487,49.303402,-124.088909,4850.90,396.15,179.87,255.47,831.49,10,Moderate,2.63,0,None,523.8307


Total Cost Per Warehouse

In [6]:
query = """
SELECT Warehouse_Latitude, Warehouse_Longitude, SUM(Shipping_Cost) AS Total_Shipping_Cost
FROM shipments
GROUP BY Warehouse_Latitude, Warehouse_Longitude
ORDER BY Total_Shipping_Cost DESC;
"""
pd.read_sql(query, conn)


,Warehouse_Latitude,Warehouse_Longitude,Total_Shipping_Cost
0,33.410482,-111.115688,499.86
1,31.301032,-115.205385,498.39
2,38.638900,-84.297640,493.89
3,42.465963,-115.461059,490.42
4,35.842893,-107.279861,485.56
...,...,...,...
95,33.636499,-94.663850,41.70
96,32.157829,-88.804978,39.62
97,33.919657,-89.535215,33.82
98,33.668090,-85.141576,32.66


Identify late shipments due to weather impact

In [22]:
query = """
SELECT Shipment_ID, Delivery_Time_Days, Weather_Impact, Traffic_Delay_Hours
FROM shipments
WHERE Weather_Impact IN ('Moderate', 'Severe') 
AND Traffic_Delay_Hours > 24;
"""
pd.read_sql(query, conn)


,Shipment_ID,Delivery_Time_Days,Weather_Impact,Traffic_Delay_Hours
0,2001,8,Moderate,37.00
1,2011,8,Moderate,43.94
2,2020,10,Severe,25.91
3,2022,4,Moderate,40.39
4,2031,5,Moderate,33.84
5,2037,9,Moderate,46.45
6,2044,10,Moderate,33.15
7,2045,11,Moderate,25.02
8,2048,7,Moderate,36.40
9,2049,2,Moderate,30.67


 Monthly Shipping Costs Pivot Table

In [12]:
query = """
SELECT strftime('%Y-%m', Order_Date) AS Order_Month,
       Warehouse_Latitude, Warehouse_Longitude, 
       SUM(Shipping_Cost) AS Total_Cost
FROM shipments
GROUP BY Order_Month, Warehouse_Latitude, Warehouse_Longitude
ORDER BY Order_Month;
"""
pd.read_sql(query, conn)


,Order_Month,Warehouse_Latitude,Warehouse_Longitude,Total_Cost
0,2023-01,30.411690,-108.409942,314.44
1,2023-01,30.929008,-99.570108,435.07
2,2023-01,31.161672,-103.584683,457.25
3,2023-01,32.789877,-84.156348,202.83
4,2023-01,33.119890,-110.028311,98.06
...,...,...,...,...
95,2023-04,40.454657,-90.961773,340.43
96,2023-04,41.225544,-94.675942,185.92
97,2023-04,44.264896,-84.111370,327.62
98,2023-04,45.215701,-83.983278,107.30


Pivot: Weather Impact vs. Shipping Delays

In [13]:
query = """
SELECT Weather_Impact, 
       AVG(Traffic_Delay_Hours) AS Avg_Delay_Hours
FROM shipments
GROUP BY Weather_Impact
ORDER BY Weather_Impact;
"""
pd.read_sql(query, conn)


,Weather_Impact,Avg_Delay_Hours
0,None,27.754909
1,Moderate,23.227838
2,Severe,15.580000


Flag High-Cost Anomalies

In [19]:
query = """
UPDATE shipments
SET Shipment_Anomaly = 'High Cost'
WHERE Total_Cost > (SELECT 1.5 * AVG(Total_Cost) FROM shipments);
"""
conn.execute(query)
conn.commit()


Identify Lost or Damaged Shipments

In [21]:
query = """
SELECT Shipment_ID, Shipment_Anomaly, Damaged_Shipment
FROM shipments
WHERE Shipment_Anomaly = 'Lost' OR Damaged_Shipment = 1;
"""
pd.read_sql(query, conn)


,Shipment_ID,Shipment_Anomaly,Damaged_Shipment
0,2019,High Cost,1
1,2024,High Cost,1
2,2028,None,1
3,2031,High Cost,1
4,2032,High Cost,1
5,2043,None,1
6,2054,None,1
7,2070,Cost Spike,1
8,2078,High Cost,1
9,2081,High Cost,1
